In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import numpy as np
import pandas as pd
import os
import collections
import math 
from sklearn import preprocessing
from IPython.display import display, HTML
from experiments.models import Analysis
from compound.models import Pathway, CompoundPathway, Compound
from data.models import Peak, Dataset, PeakDTSample, PeakQCSample
from fileupload.models import Sample
from collections import defaultdict
from experiments.pipelines.pipeline_rpy2 import Rpy2PipelineMetadata
from experiments.pipelines.helpers import convert_to_dataframe
from collections import OrderedDict

In [5]:
np.random.normal(0, 5, 10)

array([ 6.10469929,  1.00818342, -3.12017713,  1.0399412 , -5.73215609,
       -0.89263812, -8.20878363, -2.68912659, -4.42975124, -1.59574122])

### Sampling the p-values over a number of samples

In [211]:
sample_sets(100)

,t-stat,p-value
0,,
eighty,-6.098113,0.000827
forty,-6.013162,0.000922
twenty,-6.199353,0.000959
ten,-6.028534,0.001200
six,-5.842298,0.001701
four,-6.087026,0.001803
two,-5.373935,0.004370


In [209]:
### Construct the fold_chage DF with standard deviation of 5

def sample_sets(num_iterations):
    
    for it in range(num_iterations):
        fc_df = construct_fc_df()
        pathway_acts = get_pathway_act(fc_df)
        
        if it == 0:   
            t_results = get_t_test(pathway_acts).sort_index(axis=0)      
        
        else:
        
            t_results = t_results + get_t_test(pathway_acts).sort_index(axis=0)      
            
            


    final = t_results/num_iterations
    new = final.sort_values(by='p-value', ascending = True)
    
    display(new)


# print "num_p ", num_pathways

# #Set up dictionary of values

# for path in range(num_pathways): #Range starts at zero
#     name = t_results.index[path]
#     t_stat = t_results.loc[name,'t-stat']
#     p_value = t_results[name, 'p-value']
    
    




### Construct a DF of 3 pathways with 2 fold changes in the data

In [20]:
def construct_fc_df():
    sample_fnames = [u'C_1', u'C_2', u'C_3', 
           u'C_4', u'Exp_1', u'Exp_2', u'Exp_3', u'Exp_4']
    
    ("NAME", "Albatross"),
    
    
    d_names =[("two",2), ("four",4), ("six",6), ("ten",10), ("twenty",20), ("forty",40), ("eighty",80)]
    data_names = OrderedDict(d_names)
    data = [12.0, 6.0, 12.0, 6.0, -17.0, -7.0, -7.0, -17.0]

    pk_samp_intensities = []
    for name, num in data_names.iteritems():
        for n in range(num):
                peak_int_list =[]
                peak_int_list.append(name) 
                data_noise = data + np.random.normal(0, 5, len(data))
                peak_int_list.extend(list(data_noise)) #The intensities of all the samples for this peak.
                pk_samp_intensities.append(peak_int_list)

    int_df = pd.DataFrame(pk_samp_intensities).set_index([0])

    int_df.columns=sample_fnames
    int_df.index.name = "peak_name"
    int_df.columns.name ="sample_name"

    scaled_data = preprocessing.scale(np.array(int_df), axis=1)

    int_df[sample_fnames] = scaled_data

    return int_df



In [21]:
df = construct_fc_df()
display (df)

sample_name,C_1,C_2,C_3,C_4,Exp_1,Exp_2,Exp_3,Exp_4
peak_name,,,,,,,,
two,1.616816,1.076644,0.328009,0.677163,-0.830360,-0.713690,-0.812113,-1.342469
two,0.590702,1.041967,1.120516,0.059278,-1.803458,-0.203672,0.516051,-1.321384
four,1.443261,0.723031,1.146965,0.364250,-1.063062,-0.274730,-1.129175,-1.210539
four,1.120310,0.472880,1.402615,0.473030,-0.842770,-1.060257,0.013940,-1.579748
four,0.546131,0.650882,1.622794,0.826206,-0.980724,-0.424896,-0.726508,-1.513886
four,1.732565,0.287547,1.146968,0.383181,-1.127017,-0.428728,-0.926029,-1.068487
six,0.888344,0.319800,1.587589,0.885680,-1.328071,-0.611121,-0.598353,-1.143869
six,0.661186,0.697073,1.348563,0.782742,-1.808617,-0.749474,-0.030427,-0.901045
six,0.259739,0.915350,1.205133,1.348793,-1.431049,-0.672123,-0.783368,-0.842476


### Get all the peaks in the "pathway" and calculate the activity matrix

### Add a percentage of Random peaks

In [113]:
pathway_acts = get_pathway_act(int_df)
print display(pathway_acts)

sample_name,Control_4.mzXML,Control_3.mzXML,Control_2.mzXML,Control_1.mzXML,2_fold_1.mzXML,2_fold_2.mzXML,2_fold_3.mzXML,2_fold_4.mzXML
Pathways,,,,,,,,
ten,-0.422877,-0.243520,-0.399049,-0.260859,0.460752,0.175297,0.181822,0.508434
twenty,-0.419073,-0.241016,-0.426045,-0.237869,0.489019,0.187841,0.174620,0.472523
six,-0.415382,-0.262792,-0.423210,-0.222618,0.494808,0.176351,0.183778,0.469066
eighty,-0.423599,-0.234972,-0.423716,-0.233783,0.482828,0.173615,0.171158,0.488469
two,-0.396473,-0.202959,-0.438029,-0.264935,0.486694,0.162112,0.149573,0.504017
forty,-0.422936,-0.242505,-0.426148,-0.229801,0.480028,0.177218,0.181190,0.482953
four,-0.430529,-0.251111,-0.408882,-0.235089,0.454877,0.191320,0.174149,0.505265


None


In [120]:
#Method to add random standardized peaks to a pathway and return the matrix with the extra rows.

random_50 = add_random_peaks(int_df, 50)
random_100 = add_random_peaks(int_df, 100)

df_50 = get_pathway_act(random_50)
df_100 = get_pathway_act(random_100)

original_act = get_pathway_act(int_df)

print "original ", int_df.shape
display (get_t_test(original_act))
print "50% random peaks ", random_50.shape
display (get_t_test(df_50))
print "100% random peaks ", random_100.shape
display (get_t_test(df_100))





original  (162, 8)


,t-stat,p-value
0,,
two,-6.623373,0.000571
six,-6.359275,0.000709
eighty,-6.322724,0.000731
four,-6.293174,0.000750
ten,-6.231929,0.000790
twenty,-6.172221,0.000831
forty,-5.993488,0.000970


50% random peaks  (243, 8)


,t-stat,p-value
0,,
six,-6.782050,0.000503
two,-6.573730,0.000594
eighty,-6.383840,0.000695
forty,-6.211289,0.000804
ten,-5.169135,0.002076
twenty,-5.120966,0.002176
four,-4.181912,0.005801


100% random peaks  (324, 8)


,t-stat,p-value
0,,
forty,-6.057797,0.000917
eighty,5.842387,0.001109
four,4.144662,0.006047
six,-4.029243,0.006886
ten,3.654587,0.010648
twenty,3.374053,0.014967
two,-1.884118,0.108534


### Set up a DF calcutating the T-test scores and the number of metabolites

In [57]:
from scipy.stats import ttest_ind
condition_1 = [u'Control_4.mzXML', u'Control_3.mzXML', u'Control_2.mzXML', u'Control_1.mzXML']
condition_2 = [u'2_fold_1.mzXML', u'2_fold_2.mzXML', u'2_fold_3.mzXML', u'2_fold_4.mzXML']


### Method to return p-value and t-test for an activity DF

In [88]:
def get_t_test(noisy_df):

    t_test_list = []

    for pathway, row in noisy_df.iterrows():
        c1 = noisy_df.loc[pathway, condition_1].values
        c2 = noisy_df.loc[pathway, condition_2].values
        path_params = [pathway] + list(ttest_ind(c1,c2))
        t_test_list.append(path_params)



    t_test_df = pd.DataFrame(t_test_list).set_index([0])
    t_test_df.columns = ['t-stat', 'p-value']

    t_test_df = t_test_df.sort_values(by='p-value', ascending = True)

    return t_test_df


### A method to add a percentage of random peaks to a peak intensity df

In [105]:
def add_random_peaks(int_df, percent):

    #For each of the pathways add random peaks
    rand_peak_list = []   
    for name in data_names:

        df_path = int_df.ix[name]
        num_peaks = math.ceil((df_path.shape[0])*(percent/100.0))

        num_samples = df_path.shape[1]
        
        #Generate random peaks for that pathway
        
        for p in range(int(num_peaks)):
            rand_peaks = []
            #data = np.random.normal(0, 1, num_samples)
            data = np.array([1,1,1,1,1,1,1,1])
            data = np.random.normal(data) #Adding random noise
            rand_peaks.append(name)
            rand_peaks.extend(list(data))
            rand_peak_list.append(rand_peaks)

    #Construct a DF for the new peak list
    ran_df = pd.DataFrame(rand_peak_list).set_index([0])
    ran_df.columns=sample_fnames
    
    #Add the random peaks to the original DF
    new_df = pd.concat([int_df, ran_df])
    
    return new_df
    
   

In [92]:
data = np.array([1,1,1,1,1,1,1,1])

### Method to retrun the peak activities given a peak DF composed of pathways

In [87]:
def get_pathway_act(int_df):
    #For all of the pathways get all of the peak IDs
    pathway_activities = []
    for name in data_names:
        pathway_peaks = int_df.ix[name] #DF selected from peak IDs.
        w, d, c = np.linalg.svd(np.array(pathway_peaks))
        pw_act_list = []
        pw_act_list.append(name)
        pw_act_list.extend(list(c[0]))

        pathway_activities.append(pw_act_list)
        
        
        activity_df = pd.DataFrame(pathway_activities).set_index([0])
        activity_df.columns=int_df.columns
        activity_df.index.name = "Pathways"


    return activity_df
